# Uitwerking oefeningen Chapter 16 API testing

* Test de API van [deckofcardsapi.com][1].  
* Maak API tests op basis van de testcases.

[1]: https://deckofcardsapi.com/

In [ ]:
%%capture
!python -m pip install --upgrade pip pytest pytest-tags ipytest requests 

import pytest
import requests
from urllib.parse import urljoin

# alleen nodig in dit notebook
import ipytest
ipytest.autoconfig()
ipytest.config()
pytest_main = lambda opts: (ipytest.run(addopts=opts), ipytest.clean())[0]

In [ ]:
# Deck of Cards API test cases

BASE_URL: str = "https://deckofcardsapi.com/"


@pytest.fixture(scope='function')
def deck_id() -> str:
    endpoint =  '/api/deck/new/'
    response = requests.get(urljoin(BASE_URL, endpoint))
    data = response.json()
    return data['deck_id']


@pytest.mark.tags('deck-api-001')
def test_een_nieuw_spel_kaarten_heeft_de_juiste_data():
    # testdata
    endpoint =  '/api/deck/new/'
    #  GET call de API endpoint
    response = requests.get(urljoin(BASE_URL, endpoint))
    assert response.ok
    data = response.json()
    #  Bekijk de JSON
    assert data
    #  Check de JSON name: deck_id
    assert isinstance(data['deck_id'], str)
    #  Check de JSON name: shuffled
    assert not data['shuffled']
    #  Check de JSON name: remaining
    assert data['remaining'] == 52

    
@pytest.mark.tags('deck-api-002')
def test_een_nieuw_spel_kaarten_is_gesorteerd(deck_id):
    # testdata
    endpoint =  f'/api/deck/{deck_id}/draw/'
    sorted_codes = [f'{v}{c}' for c in ('S', 'D', 'C', 'H') for v in ('A','2','3','4','5','6','7','8','9','0','J','Q','K')]
    first_expected_code, *rest_expected_codes = sorted_codes  # 'AS', ['2S', '3S', ... 'HK']
    #  GET call de API endpoint
    response = requests.get(urljoin(BASE_URL, endpoint))
    assert response.ok
    data = response.json()
    #  Bekijk de JSON
    assert data
    #  Check de JSON name: cards
    match data:
        case {'cards': [{'code': card_code}]}:
            assert first_expected_code == card_code
        case _:
            pytest.fail()  # fail als de data structuur niet is zoals verwacht
    #  Check de JSON name: remaining
    remaining = data['remaining']
    assert remaining == 51
    #  Vraag de overige kaarten op
    endpoint =  f'/api/deck/{deck_id}/draw/?count={remaining}'  # endpoint with query
    response = requests.get(urljoin(BASE_URL, endpoint))
    data = response.json()
    cards = data['cards']
    #  Check de volgorde van de kaarten
    for expected_code, card in zip(rest_expected_codes, cards):
        card_code = card['code']
        assert expected_code == card_code


@pytest.mark.tags('deck-api-003')
def test_een_nieuw_gehusseld_spel_kaarten_is_niet_gesorteerd():
    # testdata
    new_shuffled_deck_endpoint = '/api/deck/new/shuffle/'
    sorted_codes = [f'{v}{c}' for c in ('S', 'D', 'C', 'H') for v in ('A','2','3','4','5','6','7','8','9','0','J','Q','K')]
    #  GET call de API endpoint
    response = requests.get(urljoin(BASE_URL, new_shuffled_deck_endpoint))
    assert response.ok
    data = response.json()
    #  Bekijk de JSON 
    assert data
    #  Check de JSON name: deck_id
    assert isinstance(data['deck_id'], str)
    #  Vraag de meerdere kaarten op
    deck_id = data['deck_id']
    remaining = data['remaining']
    draw_cards_endpoint =  f'/api/deck/{deck_id}/draw/?count={remaining}'
    response = requests.get(urljoin(BASE_URL, draw_cards_endpoint))
    assert response.ok
    data = response.json()
    cards = data['cards']
    # vergelijk de verwachten kaarten met de verkregen kaarten en plaats de resultaten in een lijst
    is_ordered = []
    for expected_code, card in zip(sorted_codes, cards):
        card_code = card['code']
        is_ordered.append(card_code == expected_code)
    # check of de kaarten geen verwachte volgorde hebben
    assert not all(is_ordered)


@pytest.mark.tags('deck-api-004')
def test_een_nieuw_gesorteerd_spel_kaarten_kan_worden_gehusseld(deck_id):
    # testdata
    endpoint =  f'/api/deck/{deck_id}/draw/'
    sorted_codes = [f'{v}{c}' for c in ('S', 'D', 'C', 'H') for v in ('A','2','3','4','5','6','7','8','9','0','J','Q','K')]
    first_expected_code, *rest_expected_codes = sorted_codes  # 'AS', ['2S', '3S', ... 'HK']
    #  GET call de API endpoint
    response = requests.get(urljoin(BASE_URL, endpoint))
    assert response.ok
    data = response.json()
    #  Bekijk de JSON
    assert data
    #  Check de JSON name: cards
    cards = data['cards']
    card = cards.pop()
    assert card['code'] == first_expected_code
    #  GET call de API
    shuffle_remaining_cards_endpoint = f'/api/deck/{deck_id}/shuffle/?remaining=true'
    response = requests.get(urljoin(BASE_URL, shuffle_remaining_cards_endpoint))
    assert response.ok, '%s, %s - %s' % (response.status_code, response.reason, response.url)
    data = response.json()
    #  Bekijk de JSON
    assert data
    #  Check de JSON name: shuffled
    assert data['shuffled']
    #  GET call de API endpoint
    deck_id = data['deck_id']
    remaining = data['remaining']
    draw_cards_endpoint =  f'/api/deck/{deck_id}/draw/?count={remaining}'
    response = requests.get(urljoin(BASE_URL, draw_cards_endpoint))
    assert response.ok
    data = response.json()
    #  Bekijk de JSON
    assert data
    #  Check de JSON name: cards
    cards = data['cards']
    assert len(cards) == remaining
    #  Check of de cards array
    # vergelijk de verwachten kaarten met de verkregen kaarten en plaats de resultaten in een lijst
    compare_results = []
    for expected_code, card in zip(sorted_codes, cards):
        card_code = card['code']
        compare_results.append(card_code == expected_code)
    # check of de kaarten geen verwachte volgorde hebben
    assert not all(compare_results)
    

@pytest.mark.tags('deck-api-005')
def test_een_getrokken_kaart_kan_niet_twee_keer_worden_teruggelegd(deck_id):
    # testdata
    endpoint =  f'/api/deck/{deck_id}/draw/?count=2'
    #  GET call de API endpoint
    response = requests.get(urljoin(BASE_URL, endpoint))
    assert response.ok
    data = response.json()
    #  Bekijk de JSON
    assert data
    #  Check de JSON name: cards
    cards = data['cards']
    card_1, card_2, *extra_cards = cards
    assert not extra_cards
    assert card_1['code'] == 'AS'
    assert card_2['code'] == '2S'
    #  Check de JSON name: remaining
    assert data['remaining'] == 50
    #  GET call de API endpoint
    return_card_endpoint = f'/api/deck/{deck_id}/return/?cards=AS'
    response = requests.get(urljoin(BASE_URL, return_card_endpoint))
    assert response.ok
    data = response.json()
    #  Bekijk de JSON
    assert data
    #  Check de JSON name: remaining
    assert data['remaining'] == 51
    #  GET call de API endpoint
    response = requests.get(urljoin(BASE_URL, return_card_endpoint))
    assert response.ok
    data = response.json()
    #  Bekijk de JSON
    assert data
    #  Check de JSON name: remaining
    assert data['remaining'] == 51
    

pytest_main(['--verbosity=1'])